# Imports e inicializações

In [1]:
import pathlib
import tarfile
import pandas as pd
import os
import time
import gc
import pdfplumber

import cld3

import multiprocessing


import numpy as np
import json
import bz2
import argparse

from enum import Enum
from unidecode import unidecode


#from openpyxl import *
import openpyxl
from openpyxl import load_workbook
from openpyxl import Workbook

import re

##Classe com vários métodos de pre-processamento de texto em português criado pelo grupo F03
import utils.preprocessing_portuguese as preprossPT

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import nltk
from nltk.corpus import stopwords
from nltk import bigrams
from sklearn.feature_extraction.text import TfidfVectorizer

from matplotlib import pyplot as plt, rcParams, ticker



from tqdm.notebook import tqdm
#from time import time
import shutil
from shutil import rmtree
from queue import Queue
from zipfile import ZipFile
from pyunpack import Archive
from collections import defaultdict

paths = {
    'mun_doc': pathlib.Path('/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/documentos_municipios'),
    'tmp-docs': pathlib.Path('/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/documentos_municipios/tmp'),
}

root_dir = "/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data"


nome_cidade = ""

In [2]:
# List of files from file: 
# https://docs.google.com/spreadsheets/d/1-YzTYRMJwRM9V6pTPB44CYHkxxsmxyVmnqQPvSmFC6A/edit?usp=sharing
compressed_files = [
    #'288_licitacoes-pirapetinga.tar.xz',
    #'289-licitacoes-coqueiral.zip',
    #'290-licitacoes-cristais.zip',
    #'302-licitacoes-olaria.7z',
    #'304-licitacoes-passa-vinte.tar.gz',
    #'306-licitacoes-arantina.7z',
    #'353-licitacoes-ijaci.7z',
    #'381-licitacoes-sao-bento-abade.zip',
    #'385-ribeirao_vermelho.zip',
    '351-licitacoes-itamarati.zip'
]



# Métodos para caracterização dos arquivos

In [3]:
def word_cloud(txt_page, stop_words, add_path, name, page_number):
    print("Nuvem de palavras")
    
    # Create a WordCloud object
    wordcloud = WordCloud(stopwords=stop_words,
                          background_color="white", 
                          max_words=1000, contour_width=3, 
                          contour_color='steelblue', 
                          width=1920, height=1080)

    # Generate a word cloud
    #print(txt_page)
    wordcloud.generate(txt_page)

    # mostrar a imagem final
    fig, ax = plt.subplots(figsize=(10,6))
    #fig, ax = plt.subplots(figsize=(19.20,10.80))
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_axis_off()

    #plt.imshow(wordcloud);
    
    if not os.path.exists(os.path.join(root_dir, "img", add_path)):

        os.makedirs(os.path.join(root_dir, "img",add_path))
    
    #print(root_dir+"/"+"img/"+add_path+"/"+"page"+str(page_number)+"-"+name)
    
    fig.savefig(root_dir+"/"+"img/"+add_path+"/"+"page"+str(page_number)+"-"+name, bbox_inches="tight")
    
    plt.close(fig)

In [4]:
def get_text_percentage(file_name: str) -> float:
    """
    Calculate the percentage of document that is covered by (searchable) text.

    If the returned percentage of text is very low, the document is
    most likely a scanned PDF
    """
    total_page_area = 0.0
    total_text_area = 0.0

    doc = fitz.open(file_name)

    for page_num, page in enumerate(doc):
        total_page_area = total_page_area + abs(page.rect)
        text_area = 0.0
        for b in page.getTextBlocks():
            r = fitz.Rect(b[:4])  # rectangle where block text appears
            text_area = text_area + abs(r)
        total_text_area = total_text_area + text_area
    doc.close()
    return total_text_area / total_page_area

In [5]:
def update_xlsx(filename, sheet_name, page_number_file, metadata, qtd_tables, page1_tamvocab, page1_dist, page2_tamvocab, page2_dist, page3_tamvocab, page3_dist):
    
    # Criar um arquivo em excel se não existir
    # Arquivo para armazenar o resultado da caracterização
    sheet_name = sheet_name.split("/")[-1]
    print(">> Criar ou abrir arquivo para salvar resultado da caracterização")
    print(os.path.exists(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_'+str(sheet_name)+'.xlsx'))
          
    if not os.path.exists(os.path.join(root_dir, "caracterizacao")):

        os.makedirs(os.path.join(root_dir, "caracterizacao"))

    if not os.path.exists(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_'+str(sheet_name)+'.xlsx'):
        
        
        wb = openpyxl.Workbook()
        
        print("Criando um novo workbook...")
        
        ws = wb.active
        
        #std=wb.get_sheet_by_name('Sheet')
        ws = wb["Sheet"]
        #wb.remove_sheet(std)
        #std.remove(std['Sheet'])
        #print("Removendo aba padrão...")
        
        ws.title = 'Geral'
        print("Renomeando aba Geral...")
                
        ws.cell(column=1, row=ws.max_row, value="Cidade")
        ws.cell(column=2, row=ws.max_row, value="Quantidade de Arquivos Corrompidos")
        ws.cell(column=3, row=ws.max_row, value="Quantidade de Arquivos Escaneados")
        
    else:
        #Open an xlsx for reading
        print(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_'+str(sheet_name)+'.xlsx')
        wb = load_workbook(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_'+str(sheet_name)+'.xlsx')
        print("Workbook loading...")
    
    
    try:
        #Get the current Active Sheet
        ws = wb[sheet_name]
        
        print("Seleciona a aba: ", sheet_name)
    except KeyError:
        print("The worksheet '{}' does not exist for workbook '{}'. Creating one...".format(sheet_name,
                                                                                        'caracterizacao_docs_'+str(sheet_name)+'.xlsx'))
        ws = wb.create_sheet(sheet_name)
        print("Worksheet '{}' created successfully for workbook '{}'.".format(sheet_name, 'caracterizacao_docs_'+str(sheet_name)+'.xlsx'))
        
        ws.cell(column=1, row=ws.max_row, value="Nome do arquivo")
        ws.cell(column=2, row=ws.max_row, value="Quantidade de páginas")
        ws.cell(column=3, row=ws.max_row, value="Metadados")  
        ws.cell(column=4, row=ws.max_row, value="Quantidade de tabelas")
        ws.cell(column=5, row=ws.max_row, value="Pag. 1 - tamanho do vocab")
        ws.cell(column=6, row=ws.max_row, value="Pag. 1 - distribuição palavras")
        ws.cell(column=7, row=ws.max_row, value="Pag. 2 - tamanho do vocab")
        ws.cell(column=8, row=ws.max_row, value="Pag. 2 - distribuição palavras")
        ws.cell(column=9, row=ws.max_row, value="Pag. 3 - tamanho do vocab")
        ws.cell(column=10, row=ws.max_row, value="Pag. 3 - distribuição palavras")
    
    
    print(ws.max_row)
    
    linha = ws.max_row + 1
    
    ws.cell(column=1, row=linha, value=filename)
    ws.cell(column=2, row=linha, value=page_number_file)
    ws.cell(column=3, row=linha, value=str(metadata))  
    ws.cell(column=4, row=linha, value=qtd_tables)
    ws.cell(column=5, row=linha, value=page1_tamvocab)
    ws.cell(column=6, row=linha, value=str(page1_dist))
    ws.cell(column=7, row=linha, value=page2_tamvocab)
    ws.cell(column=8, row=linha, value=str(page2_dist))
    ws.cell(column=9, row=linha, value=page3_tamvocab)
    ws.cell(column=10, row=linha, value=str(page3_dist))
    
    
    print("Saving file - caracterizacao_docs_", str(sheet_name),".xlsx")
    #save the csb file
    wb.save(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_'+str(sheet_name)+'.xlsx')

In [6]:
def get_info_pdf(main_path, path, name):
    
    global nome_cidade
    
    page1_tamvocab = 0
    page1_dist = ""
    page2_tamvocab = 0
    page2_dist = ""
    page3_tamvocab = 0
    page3_dist = ""
    
    pdfMetadata = ""
    qtd_pages = 0
    
    flag_escaneado = 0
    
    txt_process = preprossPT.TextPreProcessing()
    
    #Identificar o nome da cidade para criar a pasta para ela
    add_path = main_path.split("/")[-1].split(".")[0]
    nome_cidade = add_path
    

    try: 
        
        
        with pdfplumber.open(path+name) as pdf:

                print("Arquivo: ", name)
                print("Metadados: ", pdf.metadata)
                pdfMetadata = pdf.metadata




                #Count the number os pages
                print("Quantidade de páginas: ", pdf.pages[-1].page_number)
                qtd_pages = pdf.pages[-1].page_number



                for pages in pdf.pages:                                

                    page_text = pages.extract_text()


                    #Página com texto igual a none é um indicativo de página escaneada
                    if page_text is None or cld3.get_language(page_text)[0] != 'pt':
                        print("Arquivo escaneado!!")
                        # Copia os arquivos escaneados para a pasta para verificação
                        if not os.path.exists(os.path.join(root_dir, "escaneados")):

                            os.makedirs(os.path.join(root_dir, "escaneados"))

                        # Copia os arquivos escaneados para a pasta de cada cidade
                        if not os.path.exists(os.path.join(root_dir+"/"+"escaneados", add_path)):

                            os.makedirs(os.path.join(root_dir+"/"+"escaneados", add_path))

                        shutil.copy(path+"/"+name, root_dir+"/"+"escaneados"+"/"+add_path)
                        #count_scanned += 1
                        flag_escaneado = 1
                        break
                    elif pages.page_number <= 3: #Caracterização das 3 primeiras páginas
                        print(">> Page: ", pages.page_number)
                        #print(pages.extract_text())

                        #Pre-processamento do texto

                        #Colocando todo o texto em minúsculo
                        page_text = pages.dedupe_chars().extract_text().lower()


                        page_text = txt_process.remove_pronouns(page_text)

                        page_text = txt_process.remove_reduced_or_contracted_words(page_text)

                        page_text = txt_process.remove_adverbs(page_text)

                        page_text = txt_process.remove_special_characters(page_text)

                        page_text = txt_process.remove_excessive_spaces(page_text)

                        page_text = txt_process.remove_accents(page_text)

                        page_text = txt_process.remove_stopwords(page_text)

                        page_text = txt_process.remove_symbols_from_numbers(page_text)

                        page_text = txt_process.remove_numbers(page_text)

                        page_text = txt_process.remove_urls(page_text)

                        if len(page_text) != 0:

                            #Removendo letras sozinhas no texto
                            page_text = re.sub(r'(?:^| )\w(?:$| )', ' ', page_text).strip()

                            page_text = page_text.replace("_","")

                            #Tamanho do texto com palavras repetidas
                            print("Tamanho do texto em quantidade de caracteres com palavras repetidas: ", len(page_text)) 



                            #Obtendo os itens do vocabulário
                            tokens = nltk.word_tokenize(page_text)

                            #Removendo mais stopwords
                            stop_words = set(stopwords.words('portuguese'))

                            tokens = [w for w in tokens if not w.lower() in stop_words]

                            print(tokens)


                            string_bigrams = bigrams(nltk.Text(tokens))

                            #print(tokens)

                            print("Tamanho do vocabulário: ", len(tokens))

                            #print(list(string_bigrams))

                            fdist1 = nltk.FreqDist(nltk.Text(tokens))

                            #print(fdist1.most_common(50))


                           #print(type(page_text))

                            #word_cloud(page_text, stop_words, add_path, name, pages.page_number)

                            if(pages.page_number == 1):
                                page1_tamvocab = len(tokens)
                                page1_dist = fdist1.most_common(100)
                            elif(pages.page_number == 2):
                                page2_tamvocab = len(tokens)
                                page2_dist = fdist1.most_common(100)
                            elif(pages.page_number == 3):
                                page3_tamvocab = len(tokens)
                                page3_dist = fdist1.most_common(100)


                            #Extrai as tabelas das páginas quando existem    
                            tables = pages.extract_tables()



                if not flag_escaneado:
                    if tables:
                        print("Quantidade de Tabelas: ", len(tables))
                        qtd_tables = len(tables)
                    else:
                        print("Arquivo não possui tabela")
                        qtd_tables = 0


        if not flag_escaneado:
            update_xlsx(name, add_path, qtd_pages, pdfMetadata, qtd_tables, page1_tamvocab, page1_dist, page2_tamvocab, page2_dist, page3_tamvocab, page3_dist)
                



    except: 

        # Copia os arquivos corrompidos para a pasta para verificação
        if not os.path.exists(os.path.join(root_dir, "corrompido")):

            os.makedirs(os.path.join(root_dir, "corrompido"))



        # Copia os arquivos corrompidos para a pasta de cada cidade
        if not os.path.exists(os.path.join(root_dir+"/"+"corrompido", add_path)):

            os.makedirs(os.path.join(root_dir+"/"+"corrompido", add_path))

        shutil.copy(path+"/"+name, root_dir+"/"+"corrompido"+"/"+add_path)

        print(">> Arquivo: ", name)
        print(">> FALHA: Arquivo corrompido!")

        #count_corrompido += 1
        pass
        

In [7]:
def salva_escaneado_corrompido(file, flag):
    #Registrando a quantidade de arquivos corrompidos e escaneados no arquivo
    file = str(file).split("/")[-1]
    wb = load_workbook(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_'+str(file).split("/")[-1].split(".")[0]+'.xlsx')
    ws = wb.get_sheet_by_name("Geral")
    ws = wb["Geral"]
    
    if not os.path.exists(root_dir+"/"+"corrompido"+"/"+str(file).split(".")[0]):
        count_corrompido = 0
    else:
        count_corrompido = next(os.walk(root_dir+"/"+"corrompido"+"/"+str(file).split(".")[0]))[2] #dir is your directory path as string
        count_corrompido = len(count_corrompido)
    
    if not os.path.exists(root_dir+"/"+"escaneados"+"/"+str(file).split(".")[0]):
        count_scanned = 0
    else:
        count_scanned = next(os.walk(root_dir+"/"+"escaneados"+"/"+str(file).split(".")[0]))[2] #dir is your directory path as string
        count_scanned = len(count_scanned)
    
    if flag:
        linha = ws.max_row
    else:
        linha = ws.max_row + 1

    ws.cell(column=1, row=linha, value=str(file).split(".")[0])
    ws.cell(column=2, row=linha, value=str(count_corrompido))
    ws.cell(column=3, row=linha, value=str(count_scanned))  

    print("Arquivos escaneados: ", str(count_scanned))
    print("Arquivos corrompidos: ", str(count_corrompido))

    print("Saving file info corrompidos e escaneados - "+'caracterizacao_docs_'+str(file).split("/")[-1].split(".")[0]+'.xlsx')
    #save the csb file
    wb.save(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_'+str(file).split("/")[-1].split(".")[0]+'.xlsx')
    

# Métodos para acessar os arquivos de cada cidade

In [8]:
def list_by_file_format(path, file_format):
    path = path.rstrip("/") + "/"
    
    path_aux = path+"data/files/"
    try:
        file_descriptor = [json.loads(line) for line in open(path_aux  +"file_description.jsonl").readlines()]
    except: 
        print(os.listdir(path)[0])
        path_aux = path+os.listdir(path)[0]+"/data/files/"
        file_descriptor = [json.loads(line) for line in open(path_aux  +"file_description.jsonl").readlines()]
    
    files = [item["file_name"] for item in file_descriptor if item["type"] in file_format]
    return path_aux, [f"{path}{f}".rstrip(";") for f in files]

def list_pdf(path):
    return list_by_file_format(path, ["application/pdf","pdf", "pdf;"])

In [9]:
def decompress_file(tmp_path, full_filepath, filename, extension):
    '''
    Decompress a file in a specified directory.
    '''
    
    if extension == '.zip':
        file_obj = ZipFile(full_filepath, 'r')
    elif tarfile.is_tarfile(full_filepath):
        file_obj = tarfile.open(full_filepath, 'r')
    elif extension == '.7z':
        file_obj = Archive(full_filepath)
    elif not extension:
        print('Nothing to do, file uncompressed')
        return None
    
    try:
        file_obj.extractall(tmp_path)
    except(FileNotFoundError):
        pass

# Método responsável por chamar as outras funções para caracterizar os arquivos

In [10]:
def get_all_infos_from_docs(file):
    
    
    print('process {} starting doing business'.format(file))

    # Defining a path to the compressed file
    path = os.path.join(paths['mun_doc'], file)
    # Defining a temporary path to extract the file
    #tmp_path = os.path.join(paths['mun_doc'], 'tmp'+str(time.time())[:8])
    tmp_path = os.path.join(paths['mun_doc'], 'tmp-docs')
    #print(tmp_path)
    if not os.path.exists(tmp_path):
        os.mkdir(tmp_path)
    
    tmp_path = os.path.join(tmp_path, str(file).split(".")[0])
    print(tmp_path)
    if not os.path.exists(tmp_path):
        os.mkdir(tmp_path)
    

    # Decompressing
    extension = pathlib.Path(path).suffix
    #decompress_file(tmp_path, path, file, extension)

    print("Decompressing...")

    time.sleep(5)

    tmp_path, files = list_pdf(tmp_path)
    
    for name in files:
        print(name.split("/")[-1])
        get_info_pdf(path, tmp_path, name.split("/")[-1])
    

    # Deleting the uncompressed file after we're done
    rmtree(tmp_path)
    gc.collect()

    salva_escaneado_corrompido(file, 0)#O 0 indica que é a primeira escrita no arquivo
    
        
    
        

# Método para recuperar arquivos válidos dos escaneados

In [11]:
def recupera_arquivo_valido(file):
    name_folder = str(file).split(".")[0]
    
    txt_process = preprossPT.TextPreProcessing()
    

    for arq in os.listdir(os.path.join(root_dir+"/"+"escaneados", name_folder)):
        print(root_dir+"/"+"escaneados/"+name_folder+"/"+arq)
        
        count_notPT = 0
        count_none = 0
        
        with pdfplumber.open(root_dir+"/"+"escaneados/"+name_folder+"/"+arq) as pdf:
            qtd_pages = pdf.pages[-1].page_number
            
            print(qtd_pages)

            for pages in pdf.pages:                                
                page_text = pages.extract_text()
                
                #Página com texto igual a none é um indicativo de página escaneada
                if page_text is None:
                    count_none += 1
                elif cld3.get_language(page_text)[0] != 'pt':
                    count_notPT += 1
                    
        if count_none == qtd_pages:
            print("Arquivo: ", arq, "É escaneado! Todas as páginas são none!")
        elif count_notPT == qtd_pages:
            print("Arquivo: ", arq, "É escaneado! Todas as páginas não estão em pt!")
        else:
            print("Arquivo: ", arq, "NÃO É escaneado!")
            
            #try:
            for pages in pdf.pages: 
                page_text = pages.extract_text()
                if pages.page_number <= 3 and page_text is not None:
                    page_text = pages.dedupe_chars().extract_text().lower()


                    page_text = txt_process.remove_pronouns(page_text)

                    page_text = txt_process.remove_reduced_or_contracted_words(page_text)

                    page_text = txt_process.remove_adverbs(page_text)

                    page_text = txt_process.remove_special_characters(page_text)

                    page_text = txt_process.remove_excessive_spaces(page_text)

                    page_text = txt_process.remove_accents(page_text)

                    page_text = txt_process.remove_stopwords(page_text)

                    page_text = txt_process.remove_symbols_from_numbers(page_text)

                    page_text = txt_process.remove_numbers(page_text)

                    page_text = txt_process.remove_urls(page_text)

                    if len(page_text) != 0:

                        #Removendo letras sozinhas no texto
                        page_text = re.sub(r'(?:^| )\w(?:$| )', ' ', page_text).strip()

                        page_text = page_text.replace("_","")

                        #Tamanho do texto com palavras repetidas
                        print("Tamanho do texto em quantidade de caracteres com palavras repetidas: ", len(page_text)) 



                        #Obtendo os itens do vocabulário
                        tokens = nltk.word_tokenize(page_text)

                        #Removendo mais stopwords
                        stop_words = set(stopwords.words('portuguese'))

                        tokens = [w for w in tokens if not w.lower() in stop_words]

                        print(tokens)


                        string_bigrams = bigrams(nltk.Text(tokens))

                        #print(tokens)

                        print("Tamanho do vocabulário: ", len(tokens))

                        #print(list(string_bigrams))

                        fdist1 = nltk.FreqDist(nltk.Text(tokens))


                        if(pages.page_number == 1):
                            page1_tamvocab = len(tokens)
                            page1_dist = fdist1.most_common(100)
                        elif(pages.page_number == 2):
                            page2_tamvocab = len(tokens)
                            page2_dist = fdist1.most_common(100)
                        elif(pages.page_number == 3):
                            page3_tamvocab = len(tokens)
                            page3_dist = fdist1.most_common(100)


                        #Extrai as tabelas das páginas quando existem    
                        tables = pages.extract_tables()


            update_xlsx(arq, name_folder, qtd_pages, pdf.metadata, len(tables), page1_tamvocab, page1_dist, page2_tamvocab, page2_dist, page3_tamvocab, page3_dist)


            os.remove(root_dir+"/"+"escaneados/"+name_folder+"/"+arq)
            #except:
                #print("Erro ao obter informação do arquivo!! Arquivo escaneado!")
                #pass
            
    salva_escaneado_corrompido(file, 1)#O 1 indica que estou corrigindo o os escaneados
            

# Código principal para o multiprocessamento

In [12]:
decompressed_files = ["351-licitacoes-itamarati/licitacoes_itamarati_2017", 
                      "351-licitacoes-itamarati/licitacoes_itamarati_2018", 
                      "351-licitacoes-itamarati/licitacoes_itamarati_2019", 
                      "351-licitacoes-itamarati/licitacoes_itamarati_2020"]

In [13]:
if __name__ == "__main__":
    
    
    print("Quantidade de cores: ", multiprocessing.cpu_count())
    
    

    #fire off workers
    jobs = []
    
    for file in decompressed_files:
        print("Município: ", file)
        #get_all_infos_from_docs(file)
        
        p = multiprocessing.Process(target=get_all_infos_from_docs, args = (file, ))
        jobs.append(p)
        p.start()
        #print("Multiprocessing...")
        
    for proc in jobs:
        proc.join()
        proc.close()
        
        

Quantidade de cores:  8
Município:  351-licitacoes-itamarati/licitacoes_itamarati_2017
Município:  351-licitacoes-itamarati/licitacoes_itamarati_2018
process 351-licitacoes-itamarati/licitacoes_itamarati_2017 starting doing business
Município:  351-licitacoes-itamarati/licitacoes_itamarati_2019
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/documentos_municipios/tmp-docs/351-licitacoes-itamarati/licitacoes_itamarati_2017process 351-licitacoes-itamarati/licitacoes_itamarati_2018 starting doing business
Município:  351-licitacoes-itamarati/licitacoes_itamarati_2020
Decompressing...

/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/documentos_municipios/tmp-docs/351-licitacoes-itamarati/licitacoes_itamarati_2018
process 351-licitacoes-itamarati/licitacoes_itamarati_2019 starting doing business
Decompressing...
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/documentos_municipios/tmp-docs/351-licitacoes-itamarati/licitacoes_itamarati_2019
proces

Tamanho do texto em quantidade de caracteres com palavras repetidas:  1040
['prefeitura', 'municipal', 'itamarati', 'minas', 'estado', 'minas', 'gerais', 'cnpj', 'inscricao', 'estadual', 'isento', 'prefeito', 'municipal', 'parecer', 'setor', 'juridico', 'atendendo', 'determinacoes', 'contidas', 'lei', 'federal', 'decreto', 'municipal', 'lei', 'alteracoes', 'processo', 'licitatorio', 'pregao', 'presencial', 'acordo', 'determinacoes', 'leis', 'apto', 'homologacao', 'tendo', 'vista', 'informacao', 'secretaria', 'fazenda', 'existencia', 'disponibilidade', 'recursos', 'financeiros', 'condicao', 'imposta', 'lei', 'complementar', 'setor', 'juridico', 'data', 'marco', 'homologacao', 'homologo', 'adjudico', 'processo', 'licitatorio', 'autorizando', 'compra', 'produtos', 'emissao', 'documento', 'itamarati', 'minas', 'marco', 'extrato', 'homologacao', 'pregao', 'prc', 'objeto', 'aquisicao', 'confeccao', 'impressos', 'formularios', 'graficos', 'especificados', 'termo', 'referencia', 'anexo', 'edit

Metadados:  {'Title': 'EXTRATO DE HOMOLOGAÇÃO', 'Author': 'Cliente', 'Creator': 'Microsoft® Word 2010', 'CreationDate': "D:20170703083307-03'00'", 'ModDate': "D:20170703083307-03'00'", 'Producer': 'Microsoft® Word 2010'}
Quantidade de páginas:  5
Tamanho do texto em quantidade de caracteres com palavras repetidas:  1961
Arquivo não possui tabela['copias', 'estatuto', 'ata', 'posse', 'atual', 'diretoria', 'entidade', 'registrada', 'junta', 'comercial', 'caso', 'cooperativas', 'cartorio', 'registro', 'civil', 'pessoas', 'juridicas', 'caso', 'associacoes', 'caso', 'empreendimentos', 'familiares', 'devera', 'ser', 'apresentada', 'copia', 'contrato', 'social', 'registrado', 'cartorio', 'registro', 'civil', 'pessoa', 'juridica', 'projeto', 'venda', 'generos', 'alimenticios', 'agricultura', 'familiar', 'alimentacao', 'escolar', 'prova', 'atendimento', 'requisitos', 'previstos', 'lei', 'especial', 'caso', 'limite', 'individual', 'venda', 'agricultor', 'familiar', 'empreendedor', 'familiar', 'r

['prefeitura', 'municipal', 'itamarati', 'minas', 'estado', 'minas', 'gerais', 'cnpj', 'inscricao', 'estadual', 'isento', 'municipal', 'saude', 'setor', 'administrativo', 'especificados', 'termo', 'referencia', 'anexo', 'edital', 'empresas', 'vencedoras', 'recaleo', 'recapadora', 'leopoldinense', 'ltda', 'cnpj', 'valor', 'total', 'quatorze', 'mil', 'duzentos', 'quarenta', 'reais', 'recapagem', 'pneus', 'prata', 'ltda', 'cnpj', 'valor', 'total', 'vinte', 'cinco', 'mil', 'quatrocentos', 'sessenta', 'oito', 'reais', 'condicao', 'pagamento', 'pagamentos', 'serao', 'feitos', 'trinta', 'dias', 'entrega', 'produtos', 'mediante', 'apresentacao', 'nota', 'fiscal', 'aprovada', 'secretaria', 'requisitante', 'data', 'assinatura', 'homologacao', 'extrato', 'adjudicacao', 'pregao', 'prc', 'homologo', 'atos', 'praticados', 'comissao', 'permanente', 'licitacao', 'prefeitura', 'municipal', 'itamarati', 'minas', 'mg', 'procedimento', 'licitatorio', 'pregao', 'adjudicando', 'objeto', 'licitado', 'favor',

The worksheet 'licitacoes_itamarati_2018' does not exist for workbook 'caracterizacao_docs_licitacoes_itamarati_2018.xlsx'. Creating one...
Worksheet 'licitacoes_itamarati_2018' created successfully for workbook 'caracterizacao_docs_licitacoes_itamarati_2018.xlsx'.
7a550e44b4c2491a9e812b6b4f7a8f1c.pdf1

Saving file - caracterizacao_docs_ licitacoes_itamarati_2018 .xlsx
6e84c990b0fc3360ea96e8a3e9e41835.pdf
Tamanho do texto em quantidade de caracteres com palavras repetidas:  1206
['prefeitura', 'municipal', 'itamarati', 'minas', 'estado', 'minas', 'gerais', 'cnpj', 'inscricao', 'estadual', 'isento', 'edital', 'pregao', 'presencial', 'processo', 'licitatorio', 'modalidade', 'pregao', 'presencial', 'tipo', 'menor', 'preco', 'item', 'objeto', 'presente', 'licitacao', 'objeto', 'aquisicao', 'itens', 'padaria', 'pao', 'frances', 'pao', 'doce', 'cachorro', 'quente', 'bolo', 'fuba', 'cenoura', 'biscoitos', 'doces', 'salgados', 'leite', 'pasteurizados', 'ano', 'especificacoes', 'constantes', 'a

...

['prefeitura', 'municipal', 'itamarati', 'minas', 'estado', 'minas', 'gerais', 'cnpj', 'inscricao', 'estadual', 'isento', 'homologacao', 'pregao', 'presencial', 'prc', 'objeto', 'objeto', 'prestacao', 'servicos', 'transporte', 'escolar', 'alunos', 'residentes', 'diversas', 'localidades', 'rurais', 'escolas', 'municipais', 'itamarati', 'minas', 'atendimento', 'secretaria', 'municipal', 'educacao', 'cultura', 'municipio', 'dotacao', 'orcamentaria', 'ficha', 'ficha', 'ficha', 'ficha', 'ficha', 'ficha', 'ficha', 'ficha', 'ficha', 'ficha', 'ficha', 'empresa', 'vencedora', 'condicao', 'pagamento', 'valor', 'nilza', 'aparecida', 'pagamentos', 'serao', 'feitos', 'vinte', 'paula', 'silva', 'trinta', 'dias', 'sete', 'mil', 'reais', 'prestacao', 'servico', 'mediante', 'cnpj', 'apresentacao', 'nota', 'fiscal', 'aprovada', 'secretaria', 'requisitante', 'avenida', 'coronel', 'araujo', 'porto', 'centro', 'cep', 'itamarati', 'minas', 'mg', 'fone', 'fax', 'www', 'itamaratideminas', 'mg', 'gov', 'br', '

['prefeitura', 'municipal', 'itamarati', 'minas', 'estado', 'minas', 'gerais', 'cnpj', 'inscricao', 'estadual', 'isento', 'homologacao', 'adjudicacao', 'atendendo', 'determinacoes', 'contidas', 'lei', 'federal', 'alteracoes', 'prc', 'pregao', 'presencial', 'acordo', 'determinacoes', 'lei', 'apto', 'homologacao', 'adjudicacao', 'tendo', 'vista', 'informacao', 'secretaria', 'fazenda', 'existencia', 'disponibilidade', 'recursos', 'financeiros', 'condicao', 'imposta', 'lei', 'complementar', 'autorizo', 'emissao', 'documento', 'itamarati', 'minas', 'fevereiro', 'hamilton', 'moura', 'filho', 'prefeito', 'municipal', 'homologacao', 'homologo', 'adjudico', 'processo', 'licitatorio', 'autorizando', 'compra', 'produtos', 'emissao', 'documento', 'itamarati', 'minas', 'fevereiro', 'avenida', 'coronel', 'araujo', 'porto', 'centro', 'cep', 'itamarati', 'minas', 'mg', 'fone', 'fax', 'www', 'itamaratideminas', 'mg', 'gov', 'br', 'mail', 'gabinete', 'itamaratideminas', 'mg', 'gov', 'br']
Tamanho do voc

['prefeitura', 'municipal', 'itamarati', 'minas', 'estado', 'minas', 'gerais', 'cnpj', 'inscricao', 'estadual', 'isento', 'saude', 'setor', 'administrativo', 'secretaria', 'obras', 'servicos', 'urbanos', 'secretaria', 'municipal', 'agricultura', 'turismo', 'meio', 'ambiente', 'comercio', 'secretaria', 'municipal', 'acao', 'social', 'desenvolvimento', 'economico', 'municipio', 'especificacoes', 'constantes', 'anexo', 'parte', 'integrante', 'edital', 'produtos', 'licitados', 'deverao', 'ser', 'entregues', 'acordo', 'solicitacao', 'setor', 'responsavel', 'produtos', 'objeto', 'licitacao', 'deverao', 'estar', 'normas', 'tecnicas', 'aplicaveis', 'ficando', 'desde', 'estabelecido', 'serao', 'aceitos', 'exame', 'tecnico', 'efetuado', 'servidor', 'habilitado', 'indicado', 'fim', 'caso', 'satisfacam', 'especificacoes', 'exigidas', 'serao', 'aceitos', 'devendo', 'ser', 'retirados', 'fornecedor', 'prazo', 'quarenta', 'oito', 'horas', 'contados', 'partir', 'notificacao', 'clausula', 'terceira', 'l

/dados01/workspace/ufmg.f01dcc/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  """


Arquivos escaneados:  181
Arquivos corrompidos:  0
Saving file info corrompidos e escaneados - caracterizacao_docs_licitacoes_itamarati_2019.xlsx


In [14]:
#fire off workers
jobs = []
for file in decompressed_files:
    print("Recuperando escaneados município: ", file)


    p = multiprocessing.Process(target=recupera_arquivo_valido, args = (file, ))
    jobs.append(p)
    p.start()


for proc in jobs:
    proc.join()
    proc.close()

Recuperando escaneados município:  351-licitacoes-itamarati/licitacoes_itamarati_2017
Recuperando escaneados município:  351-licitacoes-itamarati/licitacoes_itamarati_2018
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2017/7a550e44b4c2491a9e812b6b4f7a8f1c.pdf
Recuperando escaneados município:  351-licitacoes-itamarati/licitacoes_itamarati_2019
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2018/bb842259f16156437f4228f4d84f9f15.pdf
Recuperando escaneados município:  351-licitacoes-itamarati/licitacoes_itamarati_2020
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2019/25d53a6dfa84734368ed91502eb9d098.pdf
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2020/adeaa82c8457a1d0c1b55d15ac5ada05.pdf
6
5
4
Ar

/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2020/a6c1465daf555e2964e40a3c7b8c2eef.pdf
4
Arquivo:  fac0e6b167453a81f94c63d1ad80c663.pdf É escaneado! Todas as páginas são none!
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2018/f7bb59a33e7eef7f5467ce8b1a4631b6.pdf
5
4
5
Arquivo:  a6c1465daf555e2964e40a3c7b8c2eef.pdf É escaneado! Todas as páginas são none!
8/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2020/f19abf8827dd74087a90aff2b4b11f5d.pdf

Arquivo:  ecebb76fa51ea188a20440661e195662.pdf É escaneado! Todas as páginas são none!
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2019/aa8d10144d9e2cb90b640ff84bbcc64b.pdf
Arquivo:  f7bb59a33e7eef7f5467ce8b1a4631b6.pdf É escaneado! Todas as páginas são 

/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2018/4f85e0f421c4bbbe064a6e557b28f678.pdf
5
4
Arquivo:  124e8cdb3be32f652afd7c3e20ea80d0.pdf É escaneado! Todas as páginas são none!
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2020/f8e0b37e596827ba435561816ec0eef4.pdf
4
Arquivo:  e7e2b7bd82d0ac9e835ae0f1e9162233.pdf É escaneado! Todas as páginas são none!Arquivo: 
 /dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2019/79f9be5070828107c625146bc09dfc41.pdf45d6158e65751e64ebcb51254c76a776.pdf 
É escaneado! Todas as páginas são none!
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2017/d92ed6d464bf6ad147488a5a7494a977.pdf
4
Arquivo:  4f85e0f421c4bbbe064a6e557b28f678.pdf É escaneado! Todas as páginas são no

/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2017/d8be0494fe4fb0800c03e1f7e0bbb92e.pdf
Arquivo:  63387f24e1b24d505677f4604ef47ff0.pdf É escaneado! Todas as páginas são none!
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2020/1b972f6ac83dd0bea52e6a30dfec5e75.pdf
Arquivo:  8c6e326398d29e0cfa653fbd93ecafb8.pdf É escaneado! Todas as páginas são none!
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2019/89805272bcaf218af3929ff213fd9119.pdf
Arquivo:  e102510404f1d5be32974e19b3c701a8.pdf É escaneado! Todas as páginas são none!
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2018/09ac9845853449fc37ec77ccf623f8ff.pdf
5
4
4
5
Arquivo:  1b972f6ac83dd0bea52e6a30dfec5e75.pdf É escaneado! Todas as páginas são no

/dados01/workspace/ufmg.f01dcc/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  """


...

Arquivo:  4aa0ebe56b05db5a3f2c698730a4d0b2.pdf É escaneado! Todas as páginas são none!
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2019/bc8e5da95f17243dcd422faf21ecdf84.pdf
5
4
Arquivo:  bc8e5da95f17243dcd422faf21ecdf84.pdf É escaneado! Todas as páginas são none!
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2019/acd6b3f9908a60b4602f830e50b0c8df.pdf
Arquivo:  2d3c3e52243c3ab2735ff293e42e1656.pdf É escaneado! Todas as páginas são none!
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2017/b2ea0f4d5ac405a2704226a427e55c54.pdf
5
5
Arquivo:  b2ea0f4d5ac405a2704226a427e55c54.pdf É escaneado! Todas as páginas são none!
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2017/c09b134725aac486b7bf89eabcc90c33.

/dados01/workspace/ufmg.f01dcc/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  """


Arquivos escaneados:  173
Arquivos corrompidos:  0
Saving file info corrompidos e escaneados - caracterizacao_docs_licitacoes_itamarati_2017.xlsx
5
Arquivo:  20743c0e128ae60f79bd824e442b0703.pdf É escaneado! Todas as páginas são none!
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2019/272c1856300d02a9f091b66dc9c168c7.pdf
4
Arquivo:  272c1856300d02a9f091b66dc9c168c7.pdf É escaneado! Todas as páginas são none!
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2019/2b934b5e360d69f7fc36996967591f52.pdf
7
Arquivo:  2b934b5e360d69f7fc36996967591f52.pdf É escaneado! Todas as páginas são none!
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/escaneados/351-licitacoes-itamarati/licitacoes_itamarati_2019/78574851ed09e209b464c710c816ff16.pdf
5
Arquivo:  78574851ed09e209b464c710c816ff16.pdf É escaneado! Todas as páginas são none!
/dados01/wor

/dados01/workspace/ufmg.f01dcc/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  """


Arquivos escaneados:  181
Arquivos corrompidos:  0
Saving file info corrompidos e escaneados - caracterizacao_docs_licitacoes_itamarati_2019.xlsx


In [16]:
import pandas as pd

xlsx_files = os.listdir('../data/caracterizacao/')
itamarati_files = [file for file in xlsx_files if "caracterizacao_docs_licitacoes_itamarati_20" in file]

all_dfs = []
num_files = 0
for file_name in itamarati_files:
    df = pd.read_excel("../data/caracterizacao/{}".format(file_name), sheet_name=[0,1])
    num_files += df[1].shape[0]
    all_dfs.append(df)

characterization = pd.concat([file[1] for file in all_dfs], axis=0).reset_index(drop=True)

escaneados = pd.concat([file[0] for file in all_dfs], axis=0).reset_index(drop=True)

escaneados_2 = pd.DataFrame(columns = escaneados.columns)
escaneados_2.loc[len(escaneados_2)] = ["489-licitacoes-itamarati",escaneados['Quantidade de Arquivos Corrompidos'].sum(),  escaneados['Quantidade de Arquivos Escaneados'].sum()]

with pd.ExcelWriter('../data/caracterizacao/caracterizacao_docs_351-licitacoes-itamarati.xlsx') as writer:    
    escaneados_2.to_excel(writer, sheet_name='Geral', index=False)
    characterization.to_excel(writer, sheet_name='489-licitacoes-itamarati', index=False)

# Código para salvar os dados dos arquivos de cada cidade em um único arquivo

In [50]:
# The below method is used to read data from an active worksheet and store it in memory.
def reader(file, name_sheet):
    #abs_file = os.path.join(path, file)
    wb_cidade = load_workbook(file)
    
    wb_sheet = wb_cidade[name_sheet]
    rows = []
    for row in wb_sheet.iter_rows(min_row=1):
        row_data = []
        for cell in row:
            row_data.append(cell.value)
        # Creating a list of lists, where each list contain a typical row's data
        rows.append(row_data)
    return rows

In [59]:
files

['caracterizacao_docs_290-licitacoes-cristais.xlsx',
 'caracterizacao_docs_304-licitacoes-passa-vinte.xlsx',
 'caracterizacao_docs_381-licitacoes-sao-bento-abade.xlsx',
 'caracterizacao_docs_289-licitacoes-coqueiral.xlsx',
 'caracterizacao_docs_353-licitacoes-ijaci.xlsx',
 'caracterizacao_docs_288_licitacoes-pirapetinga.xlsx',
 'caracterizacao_docs_385-ribeirao_vermelho.xlsx',
 'caracterizacao_docs_302-licitacoes-olaria.xlsx',
 'caracterizacao_docs_306-licitacoes-arantina.xlsx',
 'caracterizacao_docs_351-licitacoes-itamarati.xlsx']

In [60]:
for file in os.walk(root_dir+"/"+"caracterizacao"):
    files = file[2]
    break ##Para não pegar do cache

    
if not os.path.exists(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_geral.xlsx'):
        
    wb = Workbook()

    print("Criando um novo workbook...")

    ws = wb.active

    ws = wb["Sheet"]

    ws.title = 'Geral'
    print("Renomeando aba Geral...")


    ws.cell(column=1, row=ws.max_row, value="Cidade")
    ws.cell(column=2, row=ws.max_row, value="Quantidade de Arquivos Corrompidos")
    ws.cell(column=3, row=ws.max_row, value="Quantidade de Arquivos Escaneados")
    
    wb.save(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_geral.xlsx')
       
else:
    #Open an xlsx for reading
    wb = load_workbook(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_geral.xlsx')
    ws = wb["Geral"]

    
#Valores da aba Geral
for f in files:
    
    print(f)
    
    if f != 'caracterizacao_docs_geral.xlsx':
    
        linha = ws.max_row + 1
        
        wb_cidade = load_workbook(root_dir+"/"+"caracterizacao"+"/"+f)

        ws_cidade = wb_cidade["Geral"]
        
        ws = wb["Geral"]

        print(ws_cidade['A2'].value)
        print(ws_cidade['B2'].value)
        print(ws_cidade['C2'].value)



        ws.cell(column=1, row=linha, value=ws_cidade['A2'].value)
        ws.cell(column=2, row=linha, value=ws_cidade['B2'].value)
        ws.cell(column=3, row=linha, value=ws_cidade['C2'].value)
        
        #wb.save(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_geral.xlsx')
    
#files = os.listdir(root_dir+"/"+"caracterizacao")
for f in files:
    
    print(f)
    
    if f != 'caracterizacao_docs_geral.xlsx':
        
        wb_cidade = load_workbook(root_dir+"/"+"caracterizacao"+"/"+f)
        
        ws_cidade = wb_cidade["Geral"]
        
        print("Copiando aba files cidade...", ws_cidade['A2'].value)

        rows = reader(root_dir+"/"+"caracterizacao"+"/"+f, ws_cidade['A2'].value)

        ws = wb.create_sheet(ws_cidade['A2'].value)

        for row in rows:
            ws.append(row)


        wb.save(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_geral.xlsx')
        
    


Criando um novo workbook...
Renomeando aba Geral...
caracterizacao_docs_290-licitacoes-cristais.xlsx
290-licitacoes-cristais
2
449
caracterizacao_docs_304-licitacoes-passa-vinte.xlsx
304-licitacoes-passa-vinte
0
26
caracterizacao_docs_381-licitacoes-sao-bento-abade.xlsx
381-licitacoes-sao-bento-abade
0
3
caracterizacao_docs_289-licitacoes-coqueiral.xlsx
289-licitacoes-coqueiral
0
41
caracterizacao_docs_353-licitacoes-ijaci.xlsx
353-licitacoes-ijaci
0
53
caracterizacao_docs_288_licitacoes-pirapetinga.xlsx
288_licitacoes-pirapetinga
2
217
caracterizacao_docs_385-ribeirao_vermelho.xlsx
385-ribeirao_vermelho
0
512
caracterizacao_docs_302-licitacoes-olaria.xlsx
302-licitacoes-olaria
1
9
caracterizacao_docs_306-licitacoes-arantina.xlsx
306-licitacoes-arantina
0
609
caracterizacao_docs_351-licitacoes-itamarati.xlsx
489-licitacoes-itamarati
0
536
caracterizacao_docs_290-licitacoes-cristais.xlsx
Copiando aba files cidade... 290-licitacoes-cristais
caracterizacao_docs_304-licitacoes-passa-vinte.